In [1]:
# Imports
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
def seed_all(seed=42):
    """
    Sets the numpy and torch random seed.
    """
    np.random.seed(seed)
    torch.random.seed = seed

seed_all()

In [3]:
# Create some X data
X = np.random.uniform(0, 10, 100)

In [4]:
# Define the slope (m), bias (b), and some noise we want to add to X to make y
m = 3
b = 1.8
noise = np.random.normal(scale=3, size=100)

# Exercise 2.1

Based on your knowledge of linear regression, please use the variables `X`, `m`, `b`, and `noise` to generate a new variable `y`.

<!-- startquestion -->

In [5]:
y = ...

In [7]:
fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlabel('X')
ax.set_ylabel('y')
ax.scatter(X, y)

# Exercise 2.2

Please complete the `mse` function below.
There is a sanity check implemented that shoudl let you know if you've implemented the function correctly.

<!-- startquestion -->

In [8]:
# Define MSE
def mse(predictions:torch.Tensor, actuals:torch.Tensor) -> torch.Tensor:
    # your code here
    raise NotImplementedError("Implement MSE, then remove this line")

In [10]:
ys = torch.tensor([1,2,3])
yhats = torch.tensor([1.1, 2.1, 3.1])

In [11]:
assert mse(ys, yhats) == 0.01

In [12]:
print(mse(ys, yhats))
print(mse(ys, yhats).numpy())

In [14]:
assert torch.allclose(mse(ys, yhats), torch.tensor(0.01))

In [15]:
# Now that we've defined MSE, let's just use Torch's.
mse_loss = nn.MSELoss()
mse_loss(ys, yhats)

In [16]:
# We can also use the functional API to calculate MSE
F.mse_loss(ys, yhats)

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Exercise 2.3

In the exercise below, use the notebook to fit a `LinearRegression` model. 
We will inspect the parameters (the coefficient and intercept), and calculate the mean squared error.
In the cells below, complete the exercise to determine the coefficient and intercept learned by a `LinearRegression`.

<!-- startquestion -->

In [18]:
# your code here
lr = ...

In [20]:
# Display the slope and intercept
lr.coef_, lr.intercept_

In [21]:
# Calculate the mean squared error
mean_squared_error(y, lr.predict(X.reshape(-1, 1)))

In [22]:
# Plot our line of best fit
fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlabel('X')
ax.set_ylabel('y')
ax.scatter(X, y)
_x = np.arange(0, 10)
_y = _x * lr.coef_[0] + lr.intercept_
ax.plot(_x, _y, c='red', label=f"Line of best fit")
ax.legend()

In [23]:
# Because we're in torch now, let's just turn X and y into tensors.
X = torch.tensor(X)
y = torch.tensor(y)

# Exercise 2.4

In the exercise below, complete the `forward` method. 
What is the input `X`? 
How should that be transformed to the output for a linear regression?

<!-- startquestion -->

In [24]:
# Build our linear regression model
class LinReg(nn.Module):
    def __init__(self):
        super().__init__()
        # Randomly initialize 2 parameters, one for our slope and one for our bias.
        self.slope = nn.Parameter(torch.rand(1))
        self.bias = nn.Parameter(torch.rand(1))

    def forward(self, X):
        raise NotImplementedError()

In [26]:
lr = LinReg()

In [27]:
N_EPOCHS = 300
LR = 1e-3

In [28]:
slopes = []
biases = []
losses = []
_alphas = []
for i in range(N_EPOCHS):
    # Make some inferences
    yhat = lr(X)
    # Measure how bad those guesses were
    loss = F.mse_loss(yhat, y)
    if i%(N_EPOCHS/10)==0:
        print(f"Epoch {i} Train Loss: {loss:.04f}")
    # Calculate the gradient of all the parameters with respect to the loss
    loss.backward()
    # Apply the SGD update rule
    lr.slope.data.sub_(lr.slope.grad * LR)
    lr.bias.data.sub_(lr.bias.grad * LR)
    # Zero out the gradients for the next round
    lr.slope.grad.zero_()
    lr.bias.grad.zero_()

    # Record the parameters and losses so we can plot them out later
    slopes.append(float(lr.slope.data.detach().numpy()))
    biases.append(float(lr.bias.data.detach().numpy()))
    losses.append(float(loss.detach().numpy()))
    _alphas.append(i/N_EPOCHS)

In [29]:
lr.slope, lr.bias

In [30]:
fig, ax = plt.subplots(figsize=(10,10))
ax.set_xlabel('X')
ax.set_ylabel('y')
ax.scatter(X, y)
for s, b, a in zip(slopes, biases, _alphas):
    _x = np.arange(0, 10)
    _y = _x * s + b
    ax.plot(_x, _y, alpha=a, c='red', label=f"Epoch {int(a)}")


In [31]:
fig, ax = plt.subplots(figsize=(14,10))
ax.plot(losses)
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss (MSE)')
if (losses[-1] > losses[0]) | np.isnan(losses[-1]):
    ax.set_title('Diverging - BAD!')
else:
    ax.set_title('Converging - goood!')

# Exercise 2.5

Change the learning rate and number of epochs, then re-run the code up to this point, making sure to re-instantiate your model every time.
What do you notice?
What happens if you make the learning rate too large or too small?

<!-- startquestion -->

In [32]:
# Go back and re-run the code with different hyperparameters.

In [33]:
# Make yet another fake dataset
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1000, n_features=3, n_informative=2, bias=3, noise=2)

In [34]:
# No more bad habits, we need to split our data.
X_train, X_valid, y_train, y_valid = (torch.tensor(i).float() for i in train_test_split(X, y, test_size=0.1, random_state=42))

# Exercise 2.6

Complete the exercise below to create a few `nn.Parameter`s for our weights and bias.
The `weights` parameter should have the same number of elements as `X_train` has columns, and the `bias` parameter should just be a single value.
Use `torch.rand` to generate random numbers to populate the parameters.

<!-- startquestion -->

In [35]:
# Let's create some temporary weights and biases and test out our matrix operations before we build our model.
# Create a weights parameter with 1 beta per column in X
weights = ...
# Create our bias parameter
bias = ...

In [37]:
# Test out the operation we want to perform in the forward pass
torch.matmul(X_train[:10], weights) + bias

In [38]:
# FYI: @ does the same thing as matmul in this context and is easier
X_train[:10]@weights + bias

In [39]:
# Sanity check: different implementations of our forward pass are the same
assert (X_train@weights + bias == torch.matmul(X_train, weights) + bias).all()

In [40]:
# Let's make our model
class LinRegMulti(nn.Module):
    def __init__(self, n_cols):
        super().__init__()
        self.n_cols = n_cols

        self.weights = nn.Parameter(torch.rand(self.n_cols))
        self.bias = nn.Parameter(torch.rand(1))
    
    def forward(self, X):
        return X@self.weights.T + self.bias

In [41]:
N_EPOCHS = 10000
LR = 1e-3

In [42]:
lrm = LinRegMulti(X_train.shape[1])

In [43]:
# Instead of updating each parameter individually, let's make an update rule function.
def gd_update_rule(parameters, lr):
    parameters.data.sub_(parameters.grad * lr)
    parameters.grad.zero_()

In [44]:
train_losses = []
valid_losses = []

In [45]:
for i in range(N_EPOCHS):
    yhat = lrm(X_train)
    loss = mse(yhat, y_train)
    loss.backward()
    for p in lrm.parameters():
        gd_update_rule(p, LR)
    train_losses.append(loss.detach().numpy())

    with torch.no_grad():
        yhat = lrm(X_valid)
        valid_loss = mse(yhat, y_valid)
        valid_losses.append(valid_loss.numpy())

    if i%(N_EPOCHS/10) == 0:
        print(f"Epoch {i} Train Loss: {loss:.04f}, Valid Loss: {valid_loss:.04f}")

In [46]:
EPOCHS_TO_SHOW = 2000
fig, ax = plt.subplots(figsize=(12, 12))
ax.plot(train_losses[:EPOCHS_TO_SHOW], label='Train', linewidth=3, alpha=0.5)
ax.plot(valid_losses[:EPOCHS_TO_SHOW], ls='--', label='Valid')
ax.legend()

In [47]:
lrm.weights

In [48]:
lrm.bias

In [49]:
class Linear(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.weights = nn.Parameter(torch.rand((dim_in, dim_out)))
        self.bias = nn.Parameter(torch.rand(dim_out))
    
    def forward(self, X):

        return X@self.weights + self.bias

In [50]:
# Let's compare our Linear class with nn.Linear
l1 = Linear(3, 5)
l2 = nn.Linear(3, 5)

In [51]:
l2.weight

In [52]:
l1.weights

In [53]:
# We need to make sure the weights have the same values.
# If they don't, we won't be able to compare the output.
# I'm not sure why the Linear layer's weights are transposed,
# but we'll see it doesn't matter.
l1.weights.data.copy_(l2.weight.T)
l1.bias.data.copy_(l2.bias)

In [54]:
l1(X_train[:5])

In [55]:
l2(X_train[:5])

In [56]:
assert (l1(X_train[:5]) == l2(X_train[:5])).all()

In [57]:
%%timeit 
l1(X_train[:5])

In [58]:
%%timeit
 l2(X_train[:5])

In [59]:
rng = torch.arange(-5, 5.01, 0.05)
fig, ax = plt.subplots(figsize=(14, 8))
ax.plot(rng, F.relu(rng), label='ReLU')
ax.plot(rng, torch.tanh(rng), label='tanh')
ax.plot(rng, torch.sigmoid(rng), label='sigmoid')
ax.plot(rng, F.leaky_relu(rng, negative_slope=0.01), ls='--', label='leaky ReLU')
ax.set_ylim(-1.1, 1.1)
ax.set_title('Common activation functions')
ax.legend()

In [60]:
class MultiLayerRegressor(nn.Module):
    def __init__(self, dim_in, hidden_dim):
        super().__init__()
        # self.first_layer = Linear(dim_in, hidden_dim)
        self.first_layer = nn.Linear(dim_in, hidden_dim)
        # self.second_layer = Linear(hidden_dim, 1)
        self.second_layer = nn.Linear(hidden_dim, 1)
    
    def forward(self, X):
        x = self.first_layer(X)
        # x = relu(x)
        x = F.relu(x)
        x = self.second_layer(x)
        return x

In [61]:
def multilayer_regressor(in_dim, hidden_dim):
    return nn.Sequential(
        nn.Linear(in_dim, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, 1)
    )

In [62]:
# mlr = MultiLayerRegressor(3, 4)
mlr = multilayer_regressor(3, 4)

In [63]:
train_losses = []
valid_losses = []

In [64]:
LR = 1e-3
N_EPOCHS = 20000

In [65]:
# Notice that instead of iterating through our parameters and applying
# an update rule, we're just using torch's built in SGD optimizer.
opt = optim.SGD(mlr.parameters(), lr=LR)

In [71]:
for i in range(N_EPOCHS):
    yhat = mlr(X_train).squeeze()
    loss = F.mse_loss(yhat, y_train)
    loss.backward()
    opt.step()
    opt.zero_grad()
    train_losses.append(loss.detach().numpy())

    with torch.no_grad():
        yhat = mlr(X_valid).squeeze()
        valid_loss = F.mse_loss(yhat, y_valid)
        valid_losses.append(loss.numpy())
    
    if i%(N_EPOCHS/10) == 0:
        print(f"Epoch {i} Train loss: {loss:.04f}, Valid loss: {valid_loss:.04f}")

In [73]:
idx=10000
fig, ax = plt.subplots(figsize=(12, 12))
ax.plot(train_losses[:idx], label='Train', linewidth=3, alpha=0.5)
ax.plot(valid_losses[:idx], ls='--', label='Valid')
ax.legend()

In [68]:
# Modify the code above to complete the exercise